In [1]:
import pandas as pd
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory

# TODO: Make this nicer.
df = pd.DataFrame.from_records({k:v.__dict__ for k,v in get_pretrained_saes_directory().items()}).T
df.drop(columns=["expected_var_explained", "expected_l0", "config_overrides", "conversion_func"], inplace=True)
df[df.release.str.contains("bench")] # Each row is a "release" which has multiple SAEs which may have different configs / match different hook points in a model. 

/Users/adamkarvonen/SAE_bench_template/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,release,repo_id,model,saes_map,neuronpedia_id
sae_bench_gemma-2-2b_sweep_standard_ctx128_ef2_0824,sae_bench_gemma-2-2b_sweep_standard_ctx128_ef2...,canrager/lm_sae,gemma-2b,{'blocks.3.hook_resid_post__trainer_1_step_292...,{'blocks.3.hook_resid_post__trainer_1_step_292...
sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8_0824,sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8...,canrager/lm_sae,gemma-2b,{'blocks.3.hook_resid_post__trainer_5_step_488...,{'blocks.3.hook_resid_post__trainer_5_step_488...
sae_bench_gemma-2-2b_sweep_topk_ctx128_ef2_0824,sae_bench_gemma-2-2b_sweep_topk_ctx128_ef2_0824,canrager/lm_sae,gemma-2b,{'blocks.3.hook_resid_post__trainer_2_step_976...,{'blocks.3.hook_resid_post__trainer_2_step_976...
sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824,sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824,canrager/lm_sae,gemma-2b,{'blocks.3.hook_resid_post__trainer_3': 'gemma...,"{'blocks.3.hook_resid_post__trainer_3': None, ..."
sae_bench_pythia70m_sweep_gated_ctx128_0730,sae_bench_pythia70m_sweep_gated_ctx128_0730,canrager/lm_sae,pythia-70m,{'blocks.3.hook_resid_post__trainer_0': 'pythi...,"{'blocks.3.hook_resid_post__trainer_0': None, ..."
sae_bench_pythia70m_sweep_panneal_ctx128_0730,sae_bench_pythia70m_sweep_panneal_ctx128_0730,canrager/lm_sae,pythia-70m,{'blocks.3.hook_resid_post__trainer_0': 'pythi...,"{'blocks.3.hook_resid_post__trainer_0': None, ..."
sae_bench_pythia70m_sweep_standard_ctx128_0712,sae_bench_pythia70m_sweep_standard_ctx128_0712,canrager/lm_sae,pythia-70m,{'blocks.3.hook_resid_post__trainer_0': 'pythi...,"{'blocks.3.hook_resid_post__trainer_0': None, ..."
sae_bench_pythia70m_sweep_topk_ctx128_0730,sae_bench_pythia70m_sweep_topk_ctx128_0730,canrager/lm_sae,pythia-70m,{'blocks.3.hook_resid_post__trainer_0': 'pythi...,"{'blocks.3.hook_resid_post__trainer_0': None, ..."


In [3]:
# Standard imports
import os
import torch
from tqdm import tqdm
import plotly.express as px

# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading
PORT = 8000

torch.set_grad_enabled(False)

In [4]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"


print(f"Device: {device}")

Device: mps


In [5]:
from datasets import load_dataset  
from transformer_lens import HookedTransformer
from sae_lens import SAE

model = HookedTransformer.from_pretrained("gpt2-small", device = device)

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience. 
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gpt2-small-res-jb", # see other options in sae_lens/pretrained_saes.yaml
    sae_id = "blocks.8.hook_resid_pre", # won't always be a hook point
    device = device
)

/Users/adamkarvonen/SAE_bench_template/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


/Users/adamkarvonen/SAE_bench_template/.venv/lib/python3.11/site-packages/sae_lens/sae.py:136: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [6]:
from transformer_lens.utils import tokenize_and_concatenate

dataset = load_dataset(
    path = "NeelNanda/pile-10k",
    split="train",
    streaming=False,
)

token_dataset = tokenize_and_concatenate(
    dataset= dataset,# type: ignore
    tokenizer = model.tokenizer, # type: ignore
    streaming=True,
    max_length=sae.cfg.context_size,
    add_bos_token=sae.cfg.prepend_bos,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
sae.eval()  # prevents error if we're expecting a dead neuron mask for who grads

with torch.no_grad():
    # activation store can give us tokens.
    batch_tokens = token_dataset[:32]["tokens"]
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=True)

    # Use the SAE
    feature_acts = sae.encode(cache[sae.cfg.hook_name])
    sae_out = sae.decode(feature_acts)

    # save some room
    del cache

    # ignore the bos token, get the number of features that activated in each token, averaged accross batch and position
    l0 = (feature_acts[:, 1:] > 0).float().sum(-1).detach()
    print("average l0", l0.mean().item())
    px.histogram(l0.flatten().cpu().numpy()).show()

average l0 68.78961944580078


In [8]:
from transformer_lens import utils
from functools import partial

# next we want to do a reconstruction test.
def reconstr_hook(activation, hook, sae_out):
    return sae_out


def zero_abl_hook(activation, hook):
    return torch.zeros_like(activation)


print("Orig", model(batch_tokens, return_type="loss").item())
print(
    "reconstr",
    model.run_with_hooks(
        batch_tokens,
        fwd_hooks=[
            (
                sae.cfg.hook_name,
                partial(reconstr_hook, sae_out=sae_out),
            )
        ],
        return_type="loss",
    ).item(),
)
print(
    "Zero",
    model.run_with_hooks(
        batch_tokens,
        return_type="loss",
        fwd_hooks=[(sae.cfg.hook_name, zero_abl_hook)],
    ).item(),
)

Orig 3.562199592590332
reconstr 3.764155387878418
Zero 11.14659309387207
